### 环境准备

In [49]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

### 导入数据集

In [50]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter06/train_final.csv', engine='python')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter06/test_final.csv', engine='python')


In [51]:
TRAIN_IDX=train.shape[0]
TEST_IDX = TRAIN_IDX + test.shape[0]

### 数据探索

样本观测

In [ ]:
train.shape

(50000, 146)

In [ ]:
test.shape

(50000, 146)

In [ ]:
train.columns.to_list()

['continuous_annual_inc',
 'continuous_annual_inc_joint',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_dti_joint',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status',
 'continuous_mths_since_last_delinq',
 'continuous_mths_since_last_major_derog',
 'continuous_mths_since_last_record',
 'continuous_open_acc',
 'continuous_pub_rec',
 'discrete_addr_state_1_one_hot',
 'discrete_addr_state_2_one_hot',
 'discrete_addr_state_3_one_hot',
 'discrete_addr_state_4_one_hot',
 'discrete_addr_state_5_one_hot',
 'discrete_addr_state_6_one_hot',
 'discrete_addr_state_7_one_hot',
 'discrete_addr_state_8_one_hot',
 'discrete_addr_state_9_one_hot',
 'discrete_addr_state_10_one_hot',
 'discrete_addr_state_11_one_hot',
 'discrete_addr_s

In [ ]:
train['loan_status'].unique()

array([1, 0])

In [ ]:
train['loan_status'].value_counts()

1    39788
0    10212
Name: loan_status, dtype: int64

In [ ]:
test['loan_status'].value_counts()

1    40226
0     9774
Name: loan_status, dtype: int64

In [ ]:
[x for x in train.columns.to_list() if x.startswith('loan')]

['loan_status']

In [ ]:
train.head()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,...,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
0,55000.0,NaN,0.0,5.91,NaN,679.0,675.0,3600.0,3600.0,1.0,123.03,13.99,564.0,560.0,3600.0,1,30.0,30.0,NaN,7.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,65000.0,NaN,1.0,16.06,NaN,719.0,715.0,24700.0,24700.0,4.0,820.28,11.99,699.0,695.0,24700.0,1,6.0,NaN,NaN,22.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,63000.0,71000.0,0.0,10.78,13.85,699.0,695.0,20000.0,20000.0,0.0,432.66,10.78,704.0,700.0,20000.0,1,NaN,NaN,NaN,6.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,104433.0,NaN,1.0,25.37,NaN,699.0,695.0,10400.0,10400.0,3.0,289.91,22.45,704.0,700.0,10400.0,1,12.0,NaN,NaN,12.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,34000.0,NaN,0.0,10.20,NaN,694.0,690.0,11950.0,11950.0,0.0,405.18,13.44,759.0,755.0,11950.0,1,NaN,NaN,NaN,5.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
test.head()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,...,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
0,35000.0,NaN,1.0,25.65,NaN,664.0,660.0,13000.0,13000.0,1.0,428.63,11.49,659.0,655.0,13000.0,1,3.0,3.0,NaN,12.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,50000.0,NaN,0.0,22.01,NaN,729.0,725.0,11000.0,11000.0,0.0,339.10,6.89,769.0,765.0,11000.0,1,NaN,NaN,NaN,25.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,100000.0,NaN,0.0,26.39,NaN,764.0,760.0,10000.0,10000.0,1.0,301.15,5.32,684.0,680.0,10000.0,1,NaN,NaN,NaN,23.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,55000.0,NaN,0.0,12.26,NaN,664.0,660.0,13800.0,13800.0,0.0,318.75,13.67,724.0,720.0,13800.0,1,55.0,55.0,NaN,13.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,50000.0,NaN,0.0,17.55,NaN,674.0,670.0,13550.0,13550.0,3.0,491.23,18.20,574.0,570.0,13550.0,1,29.0,82.0,NaN,7.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


### 训练Baseline

shuffle 洗牌

In [58]:
train=train.sample(frac=1)
train.head()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,...,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
27864,77500.0,NaN,0.0,22.50,NaN,689.0,685.0,14400.0,14350.0,0.0,338.66,14.48,554.0,550.0,14400.0,0,NaN,NaN,NaN,10.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18683,26000.0,NaN,0.0,21.75,NaN,684.0,680.0,8000.0,8000.0,0.0,260.55,10.64,544.0,540.0,8000.0,1,NaN,NaN,75.0,6.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
11551,71000.0,NaN,0.0,29.23,NaN,694.0,690.0,8000.0,8000.0,2.0,257.24,9.76,554.0,550.0,8000.0,0,NaN,NaN,NaN,10.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9702,28000.0,NaN,0.0,33.35,NaN,684.0,680.0,12300.0,12300.0,0.0,319.01,18.99,499.0,0.0,12300.0,0,NaN,NaN,NaN,6.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15276,80000.0,NaN,0.0,15.00,NaN,679.0,675.0,2000.0,2000.0,0.0,65.69,11.22,654.0,650.0,2000.0,1,NaN,NaN,NaN,13.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


保存shuffle后的train数据集

In [59]:
train.to_csv("train.csv", index=False)
!cp train.csv '/content/drive/MyDrive/Colab Notebooks/chapter08'

A Wrapper 一个封装

In [60]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    

In [61]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")
            

In [62]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

    def train_k_fold2(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        pred_all = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            # test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
            pred_int = (self.clf.predict(dtest, num_iteration=self.best_round) > 0.5).astype(int)
            pred_all.append(pred_int)
        # test_pred /= k_fold.n_splits
        return train_pred, pred_all, acc_result, models

In [63]:
fitter = LGBFitter(label='loan_status')

Baseline的参数设置

In [64]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
params = {'num_thread': 4, 'num_leaves': 64, 'metric': 'quantile', 'objective': 'binary', 
          'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
print('参数设置: ')
print(params)
train_pred, test_pred, error_rate, models = fitter.train_k_fold(kfold, train, test, params = params)
print('mean of error_rate : ')
print(np.mean(error_rate))

参数设置: 
{'num_thread': 4, 'num_leaves': 64, 'metric': 'quantile', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
mean of error_rate : 
0.08657999999999999


baseline的准确率

In [65]:
accuracy_score(test['loan_status'],(test_pred>0.5).astype(np.int64))

0.91268

### 构建衍生变量 is_loan_amnt_10000

探索continuous开头的字段

In [66]:
[x for x in train.columns.to_list() if x.startswith('continuous')]

['continuous_annual_inc',
 'continuous_annual_inc_joint',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_dti_joint',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'continuous_mths_since_last_delinq',
 'continuous_mths_since_last_major_derog',
 'continuous_mths_since_last_record',
 'continuous_open_acc',
 'continuous_pub_rec']

In [67]:
train['continuous_loan_amnt'].describe(percentiles=np.arange(0,1,0.1))

count    50000.00000
mean     14332.53650
std       8617.58487
min       1000.00000
0%        1000.00000
10%       4800.00000
20%       6300.00000
30%       8500.00000
40%      10000.00000
50%      12000.00000
60%      15000.00000
70%      18000.00000
80%      21000.00000
90%      28000.00000
max      35000.00000
Name: continuous_loan_amnt, dtype: float64

观察loan_status和continuous_loan_amnt之间的关系

In [68]:
train[['loan_status','continuous_loan_amnt']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_loan_amnt                ...                  
                           count          mean  ...      90%      max
loan_status                                     ...                  
0                        10212.0  15622.400118  ...  28000.0  35000.0
1                        39788.0  14001.479718  ...  27200.0  35000.0

[2 rows x 15 columns]

In [69]:
loan_amnt_10000=(train['continuous_loan_amnt']==10000).astype(np.int64)
loan_amnt_10000.value_counts()

0    46323
1     3677
Name: continuous_loan_amnt, dtype: int64

In [70]:
pd.crosstab(loan_amnt_10000,train['loan_status'])

loan_status,0,1
continuous_loan_amnt,,
0,9590,36733
1,622,3055


生成衍生列is_loan_amnt_10000，形成新的训练集和测试集

In [71]:
data1=pd.concat([train,test],axis=0)
data1.shape

(100000, 146)

In [72]:
data1['is_loan_amnt_10000']=(data1['continuous_loan_amnt']==10000).astype(np.int64)

In [73]:
data1.shape

(100000, 147)

In [74]:
data1.to_csv("data1.csv", index=False)
!cp data1.csv '/content/drive/MyDrive/Colab Notebooks/chapter08'

In [75]:
train1=data1.iloc[:TRAIN_IDX, :]
test1 = data1.iloc[TRAIN_IDX:TEST_IDX, :]

In [76]:
train1.shape

(50000, 147)

In [77]:
test1.shape

(50000, 147)

通过对num_leaves参数进行调参，训练train1并观察结果

In [78]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
for num_leaves in [12, 31, 62, 81, 127]:
  params = {'num_thread': 4, 'num_leaves': num_leaves, 'metric': 'quantile', 'objective': 'binary', 
            'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
  print('参数设置: ')
  print(params)
  train_pred, test_pred, error_rate, models = fitter.train_k_fold(kfold, train1, test1, params = params)
  print('mean of error_rate : ')
  print(np.mean(error_rate))
  print('accuracy score: ')
  print(accuracy_score(test1['loan_status'],(test_pred>0.5).astype(np.int64)))

参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'quantile', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
mean of error_rate : 
0.08732
accuracy score: 
0.91362
参数设置: 
{'num_thread': 4, 'num_leaves': 31, 'metric': 'quantile', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
mean of error_rate : 
0.08718
accuracy score: 
0.91264
参数设置: 
{'num_thread': 4, 'num_leaves': 62, 'metric': 'quantile', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction

当num_leaves=12时，accuracy scroe=0.91362 为最好结果

### 构建衍生变量 is_funded_amnt_inv_10000

In [79]:
[x for x in data1.columns.to_list() if x.find('amnt')>0]

['continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_loan_amnt',
 'is_loan_amnt_10000']

对continuous_funded_amnt_inv进行数据探索

In [80]:
train1['continuous_funded_amnt_inv'].describe(percentiles=np.arange(0,1,0.05))

count    50000.000000
mean     14325.533000
std       8612.853833
min        950.000000
0%         950.000000
5%        3000.000000
10%       4800.000000
15%       5500.000000
20%       6300.000000
25%       7750.000000
30%       8500.000000
35%      10000.000000
40%      10000.000000
45%      11600.000000
50%      12000.000000
55%      14000.000000
60%      15000.000000
65%      16000.000000
70%      18000.000000
75%      20000.000000
80%      21000.000000
85%      24000.000000
90%      27900.000000
95%      32000.000000
max      35000.000000
Name: continuous_funded_amnt_inv, dtype: float64

In [81]:
train1[['loan_status','continuous_funded_amnt_inv']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.05))

continuous_funded_amnt_inv                ...                   
                                 count          mean  ...       95%      max
loan_status                                           ...                   
0                              10212.0  15610.490110  ...  34211.25  35000.0
1                              39788.0  13995.735523  ...  31275.00  35000.0

[2 rows x 25 columns]

In [82]:
funded_amnt_inv_10000=(train1['continuous_funded_amnt_inv']==10000).astype(np.int64)
funded_amnt_inv_10000.value_counts()

0    46405
1     3595
Name: continuous_funded_amnt_inv, dtype: int64

In [83]:
pd.crosstab(funded_amnt_inv_10000,train1['loan_status'])

loan_status,0,1
continuous_funded_amnt_inv,,
0,9611,36794
1,601,2994


生成衍生变量is_funded_amnt_inv_10000， 形成新的训练集和测试集

In [84]:
data2=data1
data2['is_funded_amnt_inv_10000']=(data2['continuous_funded_amnt_inv']==10000).astype(np.int64)

In [85]:
data2.shape

(100000, 148)

In [86]:
data2['is_funded_amnt_inv_10000'].value_counts()

0    92919
1     7081
Name: is_funded_amnt_inv_10000, dtype: int64

In [87]:
data2.to_csv("data2.csv", index=False)
!cp data2.csv '/content/drive/MyDrive/Colab Notebooks/chapter08'

In [88]:
train2 = data2.iloc[:TRAIN_IDX, :]
test2 = data2.iloc[TRAIN_IDX:TEST_IDX, :]

In [89]:
train2.shape

(50000, 148)

In [90]:
test2.shape

(50000, 148)

通过对num_leaves、metric的参数进行调参，训练train2并观察结果

调整num_leaves/metric
上一轮num_leaves=62时，accuracy_score最好

In [91]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
for num_leaves in [12, 31, 62, 81]:
  for metric in ['l1', 'rmse', 'quantile', 'mape', 'binary']:
    params = {'num_thread': 4, 'num_leaves': num_leaves, 'metric': metric, 'objective': 'binary', 
              'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
    print('参数设置: ')
    print(params)
    train_pred, test_pred, error_rate, models = fitter.train_k_fold(kfold, train2, test2, params = params)
    print('mean of error_rate : ')
    print(np.mean(error_rate))
    print('accuracy score: ')
    print(accuracy_score(test2['loan_status'],(test_pred>0.5).astype(np.int64)))

参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'l1', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
mean of error_rate : 
0.0869
accuracy score: 
0.91368
参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'rmse', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
mean of error_rate : 
0.0869
accuracy score: 
0.91368
参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'quantile', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bag

从结果看，这个衍生变量提升了准确率，但不同的metric参数的结果都一致。

当num_leaves=12时，accuracy score=0.91368

当num_leaves=81时，accuracy score=0.91324

这两个提升了accuracy score


### 定义一个投票融合的stacking方法，结合fitter.train_k_fold2使用。

fitter.train_k_fold2中，保留了每个model的test_pred的[0,1]值，后续根据[0,1]值进行投票，生成accuracy score

In [92]:
def vote_func(vote_list):
    vote_df = pd.DataFrame(vote_list ,columns=['vote']).vote.value_counts()
    dict_vote = {'vote': vote_df.index, 'counts': vote_df.values}
    df_vote = pd.DataFrame(dict_vote).sort_values(by='counts', ascending=False)
    vote_result = df_vote.vote[0]
    return vote_result

对train2再次训练，通过vote的方法得出accuracy score


In [98]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
for num_leaves in [12, 81]:
  for metric in ['l1', 'rmse', 'quantile', 'mape', 'binary']:
    params = {'num_thread': 4, 'num_leaves': num_leaves, 'metric': metric, 'objective': 'binary', 
              'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
    print('参数设置: ')
    print(params)
    train_pred, test_pred, error_rate, models = fitter.train_k_fold2(kfold, train2, test2, params = params)
    vote_arr = pd.DataFrame(test_pred, index=['model-1', 'model-2', 'model-3', 'model-4', 'model-5']).T
    print(vote_arr.head())
    vote_result = list()
    for i in range(len(vote_arr.values)):
        vote_result.append(vote_func(vote_arr.values[i]))
    print(len(vote_result))
    print('accuracy score: ')
    print(accuracy_score(test2['loan_status'],vote_result))

参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'l1', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
   model-1  model-2  model-3  model-4  model-5
0        1        1        1        1        1
1        1        1        1        1        1
2        1        1        1        1        1
3        1        1        1        1        1
4        0        0        0        0        0
accuracy score: 
0.91372
参数设置: 
{'num_thread': 4, 'num_leaves': 12, 'metric': 'rmse', 'objective': 'binary', 'num_round': 1500, 'learning_rate': 0.001, 'feature_fraction': 0.5, 'bagging_fraction': 0.8}
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501
The minimum is attained in round 1501

结果记录：
---
baseline ： 0.89872
---
is_loan_amnt_10000 ： 0.89916
---
is_funded_amnt_inv_10000 ： 0.901

-------